# Project Group - 

Members: Jarrik Algera, Lorenzo Bouman, Jasper IJsselstein, Joep Neelissen, Tjerk Rijkens


Student numbers: test

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

The objective of this project is to examine the relationship between weather conditions and train disruptions in the Netherlands. Railway disruption records are combined with meteorological data to quantify the extent to which factors such as precipitation, wind, and temperature are associated with delays and service interruptions. The analysis aims to establish whether specific weather conditions systematically affect railway performance and to what degree. The geographical scale is limited to the Netherlands, and after initial data cleaning and exploratory analyis a specific corridor is chosen. The temporal scale is 2024. 

## Main question

#### *To what extent do weather conditions influence train disruptions in the Netherlands?*

### Sub-questions

*Are disruptions more frequent on days with adverse weather compared to days with mild weather?*


*Do disruption durations increase under heavy precipitation, strong wind, or temperature extremes?*


*Which weather variable shows the strongest association with disruption frequency?*


*Are there seasonal differences in how weather affects train disruptions?*


*Are there regional variations in the relationship between weather and disruptions?*

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**: N/A

**Author 2**: N/A

**Author 3**: N/A

# Data Used

## Datasets

Two primary datasets are available for this study. The first dataset contains records of train disruptions in the Netherlands between 2011 and 2024. Each record provides details on the type of disruption (for example staffing problems or a signal failure), its duration, the time of occurrence, and the affected trajectory (for example Eindhoven - Venlo, Nijmegen - Venlo). This dataset represents the dependent variable, as it directly reflects railway system performance. The dataset is obtained from Rijden de Treinen (https://www.rijdendetreinen.nl/open-data/treinstoringen#downloads). It is important to realize that not every train that is delayed or cancelled is communicated by NS as a disruption; the rule of thumb that NS uses is that a disruption is communicated when multiple trains are delayed or cancelled (i.e. a major impact of the train service). The dataset is provided as a .csv file.

The second dataset is derived from the Royal Netherlands Meteorological Institute (KNMI) (https://www.daggegevens.knmi.nl/klimatologie/uurgegevens) and provides hourly weather observations. It includes variables such as precipitation intensity, temperature, and wind speed and direction. Not every weather station measures all these variables. Therefore, it is necessary to make a selection of usable stations. These variables serve as independent factors to explain variation in railway disruptions.

# Data Pipeline

## Intended Data Analysis Pipeline

### The planned analysis will follow several stages:

The planned analysis will follow several stages:

##### 1. Data cleaning and preparation
- Align disruption data with corresponding weather observations based on time and location. Thus should be executed on an hourly basis for proper comparison. 
- Filter data to dataframe of weather data in 2024. 
- Base data from weather stations on chosen train trajectory. 
- Filter through weather stations to exclude stations without rain and temperature measurements.
- Choose appropriate parameters related to the weather data to use in analysis


##### 2. Exploratory analysis
- Compute descriptive statistics for disruption frequency and duration.
- Visualize weather variables alongside disruptions to detect initial patterns.

##### 3. Correlation analysis
- Assess associations between weather conditions and disruption frequency or duration.

##### 4. Modeling
- Apply regression models to estimate the effect of precipitation, wind, and temperature on disruption likelihood and duration.
- Investigate seasonal and regional variation in the relationship between weather and disruptions.

##### 5. Evaluation and visualization
- Present results using time series plots, scatterplots, and heatmaps, alongside statistical measures.


In [6]:
%run load_weather_data
df_weather


Voorschoten
De Kooy
Schiphol
Berkhout
Hoorn Terschelling
De Bilt
Stavoren
Lelystad
Leeuwarden
Marknesse
Deelen
Hoogeveen
Eelde
Twenthe
Vlissingen
Westdorpe
Woensdrecht
Rotterdam
Cabauw Mast
Gilze-Rijen
Eindhoven
Volkel
Ell
Maastricht


,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10N,TD,...,VV,N,U,WW,IX,M,R,S,O,Y
2,215,20240101,24,160.0,60.0,70.0,110.0,73.0,48.0,62.0,...,61.0,8.0,92.0,82.0,7,0.0,1.0,0.0,0.0,0.0
3,215,20240102,24,270.0,130.0,120.0,210.0,98.0,91.0,73.0,...,65.0,8.0,83.0,23.0,7,0.0,1.0,0.0,0.0,0.0
6,235,20240101,24,150.0,60.0,70.0,90.0,65.0,46.0,53.0,...,62.0,8.0,91.0,61.0,7,0.0,1.0,0.0,0.0,0.0
7,235,20240102,24,260.0,190.0,180.0,280.0,83.0,81.0,70.0,...,56.0,8.0,91.0,52.0,7,0.0,1.0,0.0,0.0,0.0
8,240,20240101,24,150.0,80.0,80.0,110.0,68.0,41.0,62.0,...,62.0,8.0,96.0,81.0,7,0.0,1.0,0.0,0.0,0.0
9,240,20240102,24,260.0,160.0,170.0,260.0,97.0,89.0,64.0,...,75.0,8.0,79.0,23.0,7,0.0,1.0,0.0,0.0,0.0
14,249,20240101,24,150.0,60.0,60.0,100.0,67.0,30.0,58.0,...,61.0,NaN,94.0,81.0,7,0.0,1.0,0.0,0.0,0.0
15,249,20240102,24,250.0,170.0,150.0,250.0,86.0,80.0,58.0,...,62.0,NaN,82.0,51.0,7,0.0,1.0,0.0,0.0,0.0
16,251,20240101,24,180.0,90.0,100.0,120.0,69.0,57.0,51.0,...,70.0,NaN,88.0,0.0,7,0.0,0.0,0.0,0.0,0.0
17,251,20240102,24,250.0,180.0,190.0,250.0,86.0,83.0,80.0,...,21.0,NaN,95.0,53.0,7,0.0,1.0,0.0,0.0,0.0


In [12]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np

## ZELFGEMAAKTE test df, als de weerdata in een soortgelijk format komt kunnen we locaties inerpoleren met de
## zelfgemaakte function
data = {
    'STN': [215, 235, 240, 249, 251, 260, 267],
    'LON(east)': [4.437, 4.781, 4.790, 4.979, 5.346, 5.180, 5.384],
    'LAT(north)': [52.141, 52.928, 52.318, 52.644, 53.392, 52.100, 52.898],
    'NAME': ['Voorschoten', 'De Kooy', 'Schiphol', 'Berkhout', 'Hoorn Terschelling', 'De Bilt', 'Stavoren'],
    'Temperature_C': [15.2, 14.8, 15.5, 14.9, 14.1, 15.6, 14.5],
    'Wind_Speed_kmh': [22, 28, 25, 26, 35, 19, 30]
}
df_stations = pd.DataFrame(data)


def find_closest_stations(input_lat, input_lon, stations_df):
    """Finds the 3 closest weather stations to a given lat/lon point."""
    input_location = (input_lat, input_lon)
    
    distances = stations_df.apply(
        lambda row: geodesic(input_location, (row['LAT(north)'], row['LON(east)'])).km,
        axis=1
    )
    
    df_with_dist = stations_df.copy()
    df_with_dist['distance_km'] = distances
    
    return df_with_dist.sort_values('distance_km').head(3)


def get_weighted_average_weather(closest_stations_df):
    """
    Calculates the weighted average of weather data from the closest stations
    using Inverse Distance Weighting (IDW).
    
    Returns:
        A dictionary with the interpolated weather data.
    """
    # Handle the edge case where the location is exactly at a station
    if closest_stations_df['distance_km'].iloc[0] < 0.01: # less than 10 meters
        # Return the data from the first station directly
        first_station = closest_stations_df.iloc[0]
        weather_cols = ['Temperature_C', 'Wind_Speed_kmh'] # Define weather columns
        return first_station[weather_cols].to_dict()

    # Calculate inverse distance weights
    weights = 1 / closest_stations_df['distance_km']
    sum_of_weights = np.sum(weights)
    
    weighted_averages = {}
    
    # Iterate through the weather data columns to calculate the weighted average
    weather_cols_to_average = ['Temperature_C', 'Wind_Speed_kmh']
    
    for col in weather_cols_to_average:
        weighted_sum = np.sum(closest_stations_df[col] * weights)
        weighted_average = weighted_sum / sum_of_weights
        weighted_averages[col] = weighted_average
        
    return weighted_averages


# --- EXAMPLE USAGE ---

# 1. Manually enter a location (e.g., Rotterdam Centraal)
my_lat = 51.9225
my_lon = 4.47917

# 2. Find the 3 closest stations
closest_stations = find_closest_stations(my_lat, my_lon, df_stations)

print("The 3 closest stations to your location are:")
print(closest_stations[['NAME', 'STN', 'distance_km', 'Temperature_C', 'Wind_Speed_kmh']])
print("-" * 50)

# 3. Calculate and display the interpolated weather data for that location
interpolated_weather = get_weighted_average_weather(closest_stations)

print("Interpolated weather data for your location:")
print(f"Estimated Temperature: {interpolated_weather['Temperature_C']:.2f}°C")
print(f"Estimated Wind Speed: {interpolated_weather['Wind_Speed_kmh']:.2f} km/h")

The 3 closest stations to your location are:
          NAME  STN  distance_km  Temperature_C  Wind_Speed_kmh
0  Voorschoten  215    24.483698           15.2              22
2     Schiphol  240    48.886385           15.5              25
5      De Bilt  260    52.014654           15.6              19
--------------------------------------------------
Interpolated weather data for your location:
Estimated Temperature: 15.37°C
Estimated Wind Speed: 22.05 km/h
